# 1. AWS Bedrock を使ってみよう

ハンズオンの最初の演習へようこそ！ このノートブックでは、外部でホストされたモデルを call する方法を練習します。  
この演習では、AWS の Bedrock サービスと、そこでホストされる Anthropic Claude というモデルを使います。

![image](../assets/jupypter-session-bedrock.png)

### 1.1 AWS Bedrock のクライアントをインポートしセットアップする
`boto3` は、AWS の各種サービスを操作するためのSDKのクライアントです。
 `get_bedrock_client` という関数は、AWSの[Github レポジトリ](https://github.com/aws-samples/amazon-bedrock-workshop/blob/109ed616fd14c9eb26eda9bef96eb78c490d5ef6/utils/bedrock.py#L13) で提供されています。自分の環境でこのコードを動かす場合は、AWSを自分のものに設定してください。可能であれば、環境変数に設定してください。

 #### 参考：CML の環境変数を設定する方法  

[GUIから設定する場合](https://docs.cloudera.com/machine-learning/cloud/engines/topics/ml-environment-variables.html)  
[プログラム上で設定・取得する場合](https://docs.cloudera.com/machine-learning/cloud/engines/topics/ml-accessing-environmental-variables-from-projects.html)

In [1]:
import json
import os
from typing import Optional
import boto3
from botocore.config import Config

if os.environ.get("AWS_ACCESS_KEY_ID") == "":
    os.environ["AWS_ACCESS_KEY_ID"] = "<YOUR-ACCESS-KEY-ID>"   # Replace this if running in your own environment

if os.environ.get("AWS_SECRET_ACCESS_KEY") == "":
    os.environ["AWS_SECRET_ACCESS_KEY"] = "<YOUR-SECRET-ACCESS-KEY>"   # Replace this if running in your own environment

# TODO: for a lab, can reduce some of the checks in the below function
def get_bedrock_client(
    assumed_role: Optional[str] = None,
    endpoint_url: Optional[str] = None,
    region: Optional[str] = None,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides

    Parameters
    ----------
    assumed_role :
        Optional ARN of an AWS IAM role to assume for calling the Bedrock service. If not
        specified, the current active credentials will be used.
    endpoint_url :
        Optional override for the Bedrock service API Endpoint. If setting this, it should usually
        include the protocol i.e. "https://..."
    region :
        Optional name of the AWS Region in which the service should be called (e.g. "us-east-1").
        If not specified, AWS_REGION or AWS_DEFAULT_REGION environment variable will be used.
    """
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if endpoint_url:
        client_kwargs["endpoint_url"] = endpoint_url

    bedrock_client = session.client(
        service_name="bedrock-runtime",
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

上記のコードを実行することで、クライアントが初期化され、特定のAWSのリージョンでサービスが利用できるようになります。

Then the client is initialized, binding to AWS region where Bedrock service is available. [2023年10月時点](https://aws.amazon.com/about-aws/whats-new/2023/10/amazon-bedrock-asia-pacific-tokyo-aws-region/)では, サービスが利用可能なリージョンは us-east-1, us-west-2, and ap-northeast-1. のいずれかです。ハンズオンでは、 `us-east-1` をデフォルトとして使います。この設定は、環境変数で上書きすることができます。

In [2]:
# Bedrock のクライアントを AWS のクレデンシャルでを使って初期化
# AWS 上で Assumed role やカスタムエンドポイントを使っている場合は、get_bedrock_client を見て適宜呼び方を変更してください。
if os.environ.get("AWS_DEFAULT_REGION") == "":
    os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

boto3_bedrock = get_bedrock_client(
      region=os.environ.get("AWS_DEFAULT_REGION", None))

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


### 1.3 Set desired instruction: Text Summarization

### 1.3 指示を与える

このノートブックのBedrock モデル（Anthropic の Claude）は、一般的な指示に従うテキスト生成モデルです。つまり、いくつかの指示と入力テキストを提供することで、提供された指示に従った応答を生成できます。

たとえば、基盤モデルに対して、テキストの一部を数点の箇条書きで要約するように指示することができます。モデルの指示は通常、定められたパターンに従い、使用されるモデルによって異なります。言い換えれば、異なるモデルに指示を提供する標準的な方法はありません。以下では、[Anthropicの提案された構造](https://docs.anthropic.com/claude/docs/constructing-a-prompt)に従います。たとえば、`Human:` と `Assistant:` のキーワードの使用に注目してください。これらはClaudeの基盤モデルに特有のものです。

The bedrock model shown in this notebook (Anthropic's Claude) is a general instruction-following text generation model. Meaning we can provide some instructions and input text to generate a response that will follow the instructions provided. As an example, we will provide instruction to the foundational model to summarize, in a few bullet points, a chunk of a text.  Model instructions typically follow a prescribed pattern and depend on the model used. In other words, the is no standard way to provide insturctions to different models. Below we follow [Anthropic's suggested structure](https://docs.anthropic.com/claude/docs/constructing-a-prompt). For example, note the use of keywords `Human:` and `Assistant:`. These are specific to the Claude foundational model. 

In [3]:
instruction_text = """人間: XMLタグの <text></text>  で囲まれた部分の文章をようやくしてください。文章中にない情報を追加しないでください。 
                             要約は箇条書きとし、箇条書きの数は３つ以内にして、ひとつの箇条書きが完結した文となるようにしてください。
                             要約は、「以下は与えられたテキストの簡潔な要約です」という文で初めてください。 
                    <text>{{USER_TEXT}}</text>
                    アシスタント:"""

### 1.4 インプットとなるテキストを設定し、プロンプトを完成させる

以下は、要約したいテキストです。  
このテキストと指示の長さの合計が、選択したモデルのコンテキストウィンドウのサイズに収まる必要があります。claude の場合は、約9,000語です。

In [4]:
input_text = '''機械学習は、現代のビジネスが成長し、競争力を維持するために最も重要な機能の1つとなっています。
社内プロセスの自動化から、事実上消費されるあらゆる製品の設計、作成、マーケティングプロセスの最適化まで、MLモデルは私たちの仕事や私生活のほぼすべての側面に浸透しています。
MLの開発は反復的で複雑ですが、ほとんどのMLツールは機械学習のライフサイクル全体のために構築されていないため、さらに難しくなっています。
Cloudera Data Platform上のCloudera Machine Learningは、データサイエンティストがあらゆるAIユースケースに対応する単一の統合プラットフォームでコラボレーションできるようにすることで、価値実現までの時間を短縮します。
Cloudera Machine Learningは、アジャイルな実験とプロダクションMLワークフロー向けに構築されており、データ準備からMLOps、予測レポートまで、すべてを管理します。
ライフサイクル全体を通じてミッションクリティカルなMLの課題をより迅速かつ俊敏に解決し、ビジネスにとって違いをもたらす可能性のある機会を発見します。
各MLワークスペースでは、データサイエンティストのチームが、エンタープライズ・データ・クラウド内で管理されているデータに基づいて、予測アプリケーションを構築するための機械学習モデルの開発、テスト、トレーニング、最終的なデプロイを行うことができます。
MLワークスペースは、柔軟で拡張可能なエンジンを通じて、Python、R、Scala、Sparkワークロードの完全なコンテナ化された実行をサポートします。
'''

# プロンプトの {{USER_TEXT}} 部分を上記のテキストに置き換え、プロンプトを完成させる
full_prompt = instruction_text.replace("{{USER_TEXT}}", input_text)

### 1.5 Bedrock の API リクエストを作成する

プロンプトが完成したら、Bedrockに送信するJSONペイロードを生成します。  
このAPIリクエストに必要なパラメータとフォーマットは、モデルに固有のものなので、詳細はAWS Bedrockのドキュメントを参照してください。

In [5]:
# モデルが必要とするスキーマに従うJSON
body = json.dumps({"prompt": full_prompt,
             "max_tokens_to_sample":4096,
             "temperature":0.6,
             "top_k":250,
             "top_p":1.0,
             "stop_sequences":[]
              })

# モデルのIDを指定し、JSONのペイロードを使ってモデルを Call する
modelId = 'anthropic.claude-v2:1'
response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept='application/json', contentType='application/json')
response_body = json.loads(response.get('body').read())
print("Model results successfully retreived")

Model results successfully retreived


### 1.6 結果を確認する
レスポンスボディは Claude Model API に固有のものです。詳細はAWS Bedrockのドキュメントを参照してください。

In [6]:
result = response_body.get('completion')
print(result)

 Here's a brief summary of the provided text:

- Machine learning models have become critical for businesses to grow and stay competitive today by optimizing processes and products.
- ML development is complex, and most tools aren't built for the entire machine learning lifecycle. 
- Cloudera Machine Learning aims to accelerate time-to-value by enabling collaboration on a unified platform for the entire ML lifecycle.


**(おまけの演習)** ステップ 1.4 に戻り、別の文章を入力して要約させてみましょう。どんな結果になるでしょうか？

### 1.7 まとめ

* [Cloudera AI](https://docs.cloudera.com/machine-learning/cloud/product/topics/ml-product-overview.html#cdsw_overview) は、サードパーティーのモデルと柔軟に組み合わせて利用することができます。
* Cloudera AI では、JupyterLab が エディターとしてデフォルトでサポートされており、RStudio や VSCode のような [カスタムランタイム](https://docs.cloudera.com/machine-learning/cloud/runtimes/topics/)も任意で追加することができます。
* 利用者は慣れ親しんだ開発ツールを使って、 LLM ソリューションを手軽に試すことができます。

### 次のステップ: 演習 2 に進む